# MAG Analysis

Analysing assembled and binned metagenomes generated using the [nf-core/mag](https://github.com/PhilPalmer/mag/tree/genomad) pipeline on the [Rothman dataset](https://doi.org/10.1128/AEM.01448-21) with samples from the [HTP site](https://en.wikipedia.org/wiki/Hyperion_sewage_treatment_plant)


In [1]:
import os
import pandas as pd
import yaml

from Bio import SeqIO

In [2]:
########
# Change
########
# TODO: Upload directory to AWS S3 and add code to download the data if needed
data_dir = '../mag/results_rothman_htp'

##############
# Don't change
##############

# QC
fastqc_yaml = f'{data_dir}/multiqc/multiqc_data/multiqc_fastqc.yaml'

# Assembly data
# spades_dir = f'{data_dir}/Assembly/SPAdes'
# spades_fa_gz = f'{spades_dir}/SPAdes-group-HTP_scaffolds.fasta.gz'

# Genome Binning data
binner = 'MaxBin2'
busco_tsv = f'{data_dir}/GenomeBinning/QC/busco_summary.tsv'
quast_tsv = f'{data_dir}/GenomeBinning/QC/quast_summary.tsv'
bin_tsv = f'{data_dir}/GenomeBinning/bin_summary.tsv'
bin_dir = f'{data_dir}/GenomeBinning/{binner}'
contig_depths_gz = f'{data_dir}/GenomeBinning/depths/contigs/SPAdes-group-HTP-depth.txt.gz'

# Taxonomy data
genomad_dir = f'{data_dir}/Taxonomy/geNomad'
genomad_tsv = f'{genomad_dir}/group-HTP/SPAdes-group-HTP_scaffolds_aggregated_classification/SPAdes-group-HTP_scaffolds_aggregated_classification.tsv'
gtdbtk_tsv = f'{data_dir}/Taxonomy/GTDB-Tk/gtdbtk_summary.tsv'
kraken_dir = f'{data_dir}/Taxonomy/kraken2'


In [3]:
# Define helper functions
def decompress_gz(gz_file):
    """
    Decompress a gzipped file.
    """
    file = gz_file.replace('.gz', '')
    if os.path.exists(gz_file) and not os.path.exists(file):
        !gzip -dk {gz_file}
    return file

## Load the data

Load each contig with any relevant information eg the bin/MAG and coverage etc.

Link the infomation at the sample, assembly, MAG and taxonomic levels


### Quality Control (QC)
Print the sample names and get the total number of raw reads per sample

In [4]:
# Create a dictionary containing the raw number of reads for each sample

# Read the QC data
with open(fastqc_yaml, 'r') as f:
    fastqc_data = yaml.load(f, Loader=yaml.FullLoader)

# Get the sample IDs and the total number of raw reads
sample_read_counts = {k: fastqc_data[k]['Total Sequences'] for k in fastqc_data.keys()}
sample_read_counts = {k.replace('_1',''): int(sample_read_counts[k] + sample_read_counts[k.replace('_1', '_2')]) for k in sample_read_counts.keys() if '_1' in k}
samples = list(sample_read_counts.keys())
print(samples)


['SRR14530762', 'SRR14530763', 'SRR14530764', 'SRR14530765', 'SRR14530766', 'SRR14530767', 'SRR14530769', 'SRR14530770', 'SRR14530771', 'SRR14530772', 'SRR14530880', 'SRR14530881', 'SRR14530882', 'SRR14530884', 'SRR14530885', 'SRR14530886', 'SRR14530887', 'SRR14530888', 'SRR14530889', 'SRR14530890', 'SRR14530891']


### Assemblies

[metaSPAdes](http://cab.spbu.ru/software/spades/) was used for the assembly and outputs the contigs i.e. merged reads as well as the scaffolds i.e. contigs that have been merged together

The commented code below loads the contigs, however, we will be skipping this step because the same information will be loaded using the MAGs instead


In [5]:
# # Decompress the FASTA file (if it hasn't already been decompressed)
# spades_fa = decompress_gz(spades_fa_gz)

# # Load the FASTA seqs and save them as a dict in the following format {'node_id': ['length', 'cov', 'seq']}
# seqs_dict = {int(record.id.split('_')[1]): [int(record.id.split('_')[3]), float(record.id.split('_')[5]), str(record.seq)] for record in SeqIO.parse(spades_fa, 'fasta')}

# # Create dataframe containing contigs info
# seqs_df = pd.DataFrame.from_dict(seqs_dict, orient='index', columns=['length', 'coverage', 'seq'])

# seqs_df.head()

### Load the bins/MAGs

- Two tools were used to perform metagenome binning to generate metagenome assembled genomes (MAGs) - [MetaBAT2](https://bitbucket.org/berkeleylab/metabat/src/master/) and [MaxBin2](https://sourceforge.net/projects/maxbin2/)

- And two tools were used to check for quality control (QC) of the genome bins - [Busco](https://busco.ezlab.org/) and [Quast](http://quast.sourceforge.net/quast)


Let's look the summary information of the bins/MAGs


In [6]:
bin_df = pd.read_csv(bin_tsv, sep='\t')
bin_df.head()

,Unnamed: 0,bin,Depth SRR14530762,Depth SRR14530763,Depth SRR14530764,Depth SRR14530765,Depth SRR14530766,Depth SRR14530767,Depth SRR14530769,Depth SRR14530770,...,closest_placement_ani,closest_placement_af,pplacer_taxonomy,classification_method,note,"other_related_references(genome_id,species_name,radius,ANI,AF)",msa_percent,translation_table,red_value,warnings
0,0,SPAdes-MetaBAT2-group-HTP.1.fa,0.066717,23.853900,13.459800,42.154750,10.773450,12.801900,0.002138,36.670650,...,NaN,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,NaN,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",90.19,11.0,0.956510,NaN
1,1,SPAdes-MaxBin2-group-HTP.001.fa,0.976038,0.780799,0.778258,0.558656,0.494961,0.633394,0.576577,0.502917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,SPAdes-MaxBin2-group-HTP.002.fa,0.368126,1.556870,1.357770,0.665245,0.527326,0.434845,0.108499,1.027780,...,NaN,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,NaN,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",86.76,11.0,0.956871,Genome has more than 12.5% of markers with mul...


The results from the tools show that MaxBin2 generated more genome bins and a higher % completeness than MetaBAT2


In [7]:
busco_df = pd.read_csv(busco_tsv, sep='\t')
busco_df.head()

,GenomeBin,Domain,%Complete (domain),%Complete and single-copy (domain),%Complete and duplicated (domain),%Fragmented (domain),%Missing (domain),Total number (domain),Specific lineage dataset,%Complete (specific),%Complete and single-copy (specific),%Complete and duplicated (specific),%Fragmented (specific),%Missing (specific),Total number (specific)
0,SPAdes-MaxBin2-group-HTP.002.fa,bacteria_odb10,99.2,91.1,8.1,0.0,0.8,124,betaproteobacteria_odb10,98.6,97.7,0.9,0.4,1.0,569.0
1,SPAdes-MetaBAT2-group-HTP.1.fa,bacteria_odb10,92.7,91.1,1.6,0.0,7.3,124,burkholderiales_odb10,93.3,92.9,0.4,1.2,5.5,688.0
2,SPAdes-MaxBin2-group-HTP.001.fa,bacteria_odb10,10.5,9.7,0.8,0.8,88.7,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
quast_df = pd.read_csv(quast_tsv, sep='\t')
quast_df.head()

,Assembly,# contigs (>= 0 bp),# contigs (>= 1000 bp),# contigs (>= 5000 bp),# contigs (>= 10000 bp),# contigs (>= 25000 bp),# contigs (>= 50000 bp),Total length (>= 0 bp),Total length (>= 1000 bp),Total length (>= 5000 bp),...,# contigs,Largest contig,Total length,GC (%),N50,N75,L50,L75,# N's per 100 kbp,# predicted rRNA genes
0,SPAdes-MetaBAT2-group-HTP.1.fa,54,54,45,42,31,20,3350955,3350955,3325616,...,54,429168,3350955,58.16,135708,62821,9,18,20.89,0 + 0 part
1,SPAdes-MaxBin2-group-HTP.001.fa,917,917,36,0,0,0,1736869,1736869,237600,...,917,9237,1736869,46.08,1917,1327,258,534,11.51,3 + 33 part
2,SPAdes-MaxBin2-group-HTP.002.fa,963,963,95,45,32,20,5176992,5176992,3676500,...,963,429168,5176992,56.92,58262,3791,19,143,19.32,2 + 34 part


We will therefore use the MaxBin2 results for the rest of the analysis


In [9]:
# Get the bin files
bin_files = []

for dirpath, dirnames, filenames in os.walk(bin_dir):
    bin_files.extend([os.path.join(dirpath, file) for file in filenames if file.endswith('.gz')])

bin_files

['../mag/results_rothman_htp/GenomeBinning/MaxBin2/unbinned/discarded/SPAdes-MaxBin2-group-HTP.noclass.pooled.fa.gz',
 '../mag/results_rothman_htp/GenomeBinning/MaxBin2/unbinned/discarded/SPAdes-MaxBin2-group-HTP.noclass.remaining.fa.gz',
 '../mag/results_rothman_htp/GenomeBinning/MaxBin2/bins/SPAdes-MaxBin2-group-HTP.001.fa.gz',
 '../mag/results_rothman_htp/GenomeBinning/MaxBin2/bins/SPAdes-MaxBin2-group-HTP.002.fa.gz',
 '../mag/results_rothman_htp/GenomeBinning/MaxBin2/discarded/SPAdes-MaxBin2-group-HTP.tooshort.gz']

In [10]:
bin_dfs = []

for bin_file_gz in bin_files:

    # Load the bin file and save as a dataframe
    bin_file = decompress_gz(bin_file_gz)
    bin_dict = {int(record.id.split('_')[1]): [int(record.id.split('_')[3]), float(record.id.split('_')[5]), str(record.seq)] for record in SeqIO.parse(bin_file, 'fasta')}
    bin_df = pd.DataFrame.from_dict(bin_dict, orient='index', columns=['length', 'coverage', 'seq'])

    # Add the bin ID to the dataframe
    bin_df['bin_id'] = bin_file.split('/')[-1].split('.')[1]

    # Add the dataframe to the list
    bin_dfs.append(bin_df)

# Concatenate all of the dataframes
bin_df = pd.concat(bin_dfs).sort_index()
bin_df = bin_df[~bin_df.index.duplicated(keep='first')]
bin_df.head()

,length,coverage,seq,bin_id
1,429168,132.214891,GACACCAGCAAGCGTCCGCCAAGCAAGCTTCTACGATCAAAAACTT...,002
2,214794,155.876023,CATTGGGCCCGGGGGGGGGCCACGCCCCCCCCAACCAAAAAACCGA...,002
3,204790,151.550375,GCCTGAACCTGAACATGCCGCTGTCCGATGAAATGAAGGCCTTGTA...,002
4,185392,139.905194,CCGTCAAAGAGCATCTGGTTCAGGCTCAGCACGTAGCCGCTTCGGT...,002
5,148022,144.176302,TGTCGCATGCGACAAGATCGGTACTCCCACAGGGGACTTTCACCCC...,002


## Taxonomic classification


### Virus classification

Let's load the virus classifications predicted using [geNomad](https://github.com/apcamargo/genomad) and combine this with our existing data for the contigs


In [11]:
# Load the virus classification data
vir_df = pd.read_csv(genomad_tsv, sep='\t')
vir_df.index = vir_df.seq_name.str.split('_').str[1].astype(int)
vir_df = vir_df.drop('seq_name', axis=1)
vir_df.head()

,chromosome_score,plasmid_score,virus_score
seq_name,,,
1,0.9354,0.0472,0.0175
2,0.9492,0.0370,0.0138
3,0.9312,0.0504,0.0184
4,0.9466,0.0389,0.0145
5,0.9232,0.0566,0.0202


In [12]:
# Merge the binning and taxonomy dataframes
df = pd.merge(bin_df, vir_df, left_index=True, right_index=True)
df.head()

,length,coverage,seq,bin_id,chromosome_score,plasmid_score,virus_score
1,429168,132.214891,GACACCAGCAAGCGTCCGCCAAGCAAGCTTCTACGATCAAAAACTT...,002,0.9354,0.0472,0.0175
2,214794,155.876023,CATTGGGCCCGGGGGGGGGCCACGCCCCCCCCAACCAAAAAACCGA...,002,0.9492,0.0370,0.0138
3,204790,151.550375,GCCTGAACCTGAACATGCCGCTGTCCGATGAAATGAAGGCCTTGTA...,002,0.9312,0.0504,0.0184
4,185392,139.905194,CCGTCAAAGAGCATCTGGTTCAGGCTCAGCACGTAGCCGCTTCGGT...,002,0.9466,0.0389,0.0145
5,148022,144.176302,TGTCGCATGCGACAAGATCGGTACTCCCACAGGGGACTTTCACCCC...,002,0.9232,0.0566,0.0202


### Taxonomic classification of binned genomes

Load the GTDB-Tk summary table (see [column descriptions](https://ecogenomics.github.io/GTDBTk/files/summary.tsv.html)) and combine with the existng information for the contigs


In [13]:
# Load the GTDB-Tk classification data
gtdbtk_df = pd.read_csv(gtdbtk_tsv, sep='\t')

# Filter the GTDB-Tk dataframe
gtdbtk_df = gtdbtk_df[gtdbtk_df.user_genome.str.contains(binner)]
cols = ['user_genome', 'classification', 'classification_method', 'other_related_references(genome_id,species_name,radius,ANI,AF)', 'msa_percent', 'red_value', 'warnings']
gtdbtk_df = gtdbtk_df[cols]

In [14]:
# Add the GTDB-Tk classification data to the main dataframe
gtdbtk_df['bin_id'] = gtdbtk_df.user_genome.str.split('.').str[1]
gtdbtk_df = gtdbtk_df.drop('user_genome', axis=1)

# Merge the GTDB-Tk dataframe with the main dataframe
df = pd.merge(df, gtdbtk_df, on='bin_id', how='left')

# Return index to how it was before
df.index = df.index + 1

df.head()

,length,coverage,seq,bin_id,chromosome_score,plasmid_score,virus_score,classification,classification_method,"other_related_references(genome_id,species_name,radius,ANI,AF)",msa_percent,red_value,warnings
1,429168,132.214891,GACACCAGCAAGCGTCCGCCAAGCAAGCTTCTACGATCAAAAACTT...,002,0.9354,0.0472,0.0175,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",86.76,0.956871,Genome has more than 12.5% of markers with mul...
2,214794,155.876023,CATTGGGCCCGGGGGGGGGCCACGCCCCCCCCAACCAAAAAACCGA...,002,0.9492,0.0370,0.0138,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",86.76,0.956871,Genome has more than 12.5% of markers with mul...
3,204790,151.550375,GCCTGAACCTGAACATGCCGCTGTCCGATGAAATGAAGGCCTTGTA...,002,0.9312,0.0504,0.0184,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",86.76,0.956871,Genome has more than 12.5% of markers with mul...
4,185392,139.905194,CCGTCAAAGAGCATCTGGTTCAGGCTCAGCACGTAGCCGCTTCGGT...,002,0.9466,0.0389,0.0145,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",86.76,0.956871,Genome has more than 12.5% of markers with mul...
5,148022,144.176302,TGTCGCATGCGACAAGATCGGTACTCCCACAGGGGACTTTCACCCC...,002,0.9232,0.0566,0.0202,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",86.76,0.956871,Genome has more than 12.5% of markers with mul...


### Taxonomic classification of trimmed reads

The [kraken2](https://github.com/DerrickWood/kraken2/wiki/Manual) tool was used to classify trimmed reads using the [prebuilt 8GB minikraken DB](https://zenodo.org/record/4024003#.Y4-9PdLMK0o) as provided by the Center for Computational Biology of the John Hopkins University (from 2020-03). (**Note:** Using a larger kraken database would likely improve the results by decreasing the number of unclassified reads). The outputs were tab seperated files with the following columns:  

1. `percentage` - Percentage of fragments covered by the clade rooted at this taxon
2. `num_fragments` - Number of fragments covered by the clade rooted at this taxon
3. `num_assigned` - Number of fragments assigned directly to this taxon
4. `rank_code` - A rank code, indicating (U)nclassified, (R)oot, (D)omain, (K)ingdom, (P)hylum, (C)lass, (O)rder, (F)amily, (G)enus, or (S)pecies. Taxa that are not at any of these 10 ranks have a rank code that is formed by using the rank code of the closest ancestor rank with a number indicating the distance from that rank. E.g., "G2" is a rank code indicating a taxon is between genus and species and the grandparent taxon is at the genus rank.
5. `tax_id` NCBI taxonomic ID number
6. `scientific_name` - Indented scientific name


In [15]:
kraken_dfs = []
names = ['percentage', 'num_fragments', 'num_assigned', 'rank_code', 'tax_id', 'scientific_name']

for sample in samples:
    kraken_file = f'{kraken_dir}/{sample}/kraken2_report.txt'
    kraken_df = pd.read_csv(kraken_file, sep='\t', header=None, names=names)
    kraken_df['sample_id'] = sample
    kraken_dfs.append(kraken_df)

kraken_df = pd.concat(kraken_dfs)

# Remove whitespace from the scientific name
kraken_df.scientific_name = kraken_df.scientific_name.str.strip()
kraken_df = kraken_df.reset_index()
kraken_df.head()

,index,percentage,num_fragments,num_assigned,rank_code,tax_id,scientific_name,sample_id
0,0,49.65,1776130,1776130,U,0,unclassified,SRR14530762
1,1,50.35,1801248,430,R,1,root,SRR14530762
2,2,43.78,1566074,13020,R1,131567,cellular organisms,SRR14530762
3,3,42.81,1531604,298743,D,2,Bacteria,SRR14530762
4,4,22.46,803324,115323,P,1224,Proteobacteria,SRR14530762


In [16]:
# Define vars
cols = ['total_raw_reads', 'total_processed_reads', 'classified_reads', 'unclassified_reads', 'bacterial_reads', 'viral_reads']
display_as_percent = True
vmax = None

# Generate the individual dataframes
unclass_df = kraken_df[kraken_df['rank_code'] == 'U'].groupby('sample_id').agg({'num_fragments': 'sum'}).rename(columns={'num_fragments': 'unclassified_reads'})
class_df = kraken_df[kraken_df['rank_code'] == 'R'].groupby('sample_id').agg({'num_fragments': 'sum'}).rename(columns={'num_fragments': 'classified_reads'})
bact_df = kraken_df[kraken_df['scientific_name'].str.contains('Bacteria')].groupby('sample_id').agg({'num_fragments': 'sum'}).rename(columns={'num_fragments': 'bacterial_reads'})
viral_df = kraken_df[kraken_df['scientific_name'].str.contains('Viruses')].groupby('sample_id').agg({'num_fragments': 'sum'}).rename(columns={'num_fragments': 'viral_reads'})

# Combine the dataframes and then process
summary_df = pd.concat([unclass_df, class_df, bact_df, viral_df], axis=1)
summary_df['total_raw_reads'] = [sample_read_counts[sample] for sample in summary_df.index]
summary_df['total_processed_reads'] = summary_df['unclassified_reads'] + summary_df['classified_reads']
summary_df['total_raw_reads'] = summary_df['total_raw_reads'].astype(int)
summary_df['total_processed_reads'] = summary_df['total_processed_reads'].astype(int)
summary_df.loc['total'] = summary_df.sum()
summary_df = summary_df[cols]

# Calculate percentages
if display_as_percent:
    vmax = 100
    for col in cols[2:]:
        cols[cols.index(col)] = f'{col} (%)'
        summary_df = summary_df.rename(columns={col: f'{col} (%)'})
        col = f'{col} (%)'
        summary_df[col] = summary_df[col] / summary_df['total_processed_reads'] * 100

# Display the summary dataframe with bars
summary_df.style\
    .bar(subset=cols[:2], color='#d65f5f')\
    .bar(subset=cols[2:], color='#5fba7d', vmax=vmax)\
    .format('{:,.0f}', subset=cols[:2])\
    .format('{:.2f}', subset=cols[2:])

,total_raw_reads,total_processed_reads,classified_reads (%),unclassified_reads (%),bacterial_reads (%),viral_reads (%)
sample_id,,,,,,
SRR14530762,"7,159,400","3,577,378",50.35,49.65,42.95,6.56
SRR14530763,"7,102,004","3,547,950",40.02,59.98,29.43,9.82
SRR14530764,"7,209,358","3,601,300",44.16,55.84,33.44,9.83
SRR14530765,"6,972,516","3,484,197",31.65,68.35,23.96,6.48
SRR14530766,"5,600,078","2,798,919",50.86,49.14,42.25,7.16
SRR14530767,"6,636,696","3,315,778",53.92,46.08,42.84,8.34
SRR14530769,"8,430,942","4,207,723",61.71,38.29,47.95,11.02
SRR14530770,"8,701,488","4,346,011",30.63,69.37,27.47,2.52
SRR14530771,"4,358,340","2,178,510",40.46,59.54,36.30,3.71


(Adding summary table screenshot here as the bars do not appear on GitHub)
![kraken2](data/kraken2_summary_table.png)

In [17]:
# # Display the top 50 most abundant taxa
# kraken_df = kraken_df[kraken_df.rank_code != 'U']
# kraken_df = kraken_df[kraken_df.rank_code != 'R']
# kraken_df = kraken_df.sort_values('num_fragments', ascending=False)
# kraken_df.head(50)

In [18]:
# Generate a Krona chart for all of samples - this is a bit hacky
# !docker run -v $PWD:$PWD -w $PWD quay.io/biocontainers/krona:2.7.1--pl526_5 ktUpdateTaxonomy.sh taxonomy && !ktImportTaxonomy SRR14530762/kraken2_report.txt SRR14530763/kraken2_report.txt SRR14530764/kraken2_report.txt SRR14530765/kraken2_report.txt SRR14530766/kraken2_report.txt SRR14530767/kraken2_report.txt SRR14530769/kraken2_report.txt SRR14530770/kraken2_report.txt SRR14530771/kraken2_report.txt SRR14530772/kraken2_report.txt SRR14530880/kraken2_report.txt SRR14530881/kraken2_report.txt SRR14530882/kraken2_report.txt SRR14530884/kraken2_report.txt SRR14530885/kraken2_report.txt SRR14530886/kraken2_report.txt SRR14530887/kraken2_report.txt SRR14530888/kraken2_report.txt SRR14530889/kraken2_report.txt SRR14530890/kraken2_report.txt SRR14530891/kraken2_report.txt -tax taxonomy

## Get read counts for the MAGs

### Get the sequencing depths per contig

Load the sequencing depths per contig and sample. This file is roughly equivalent to a count matrix i.e. a file mapping contig IDs (rows) and sample SRA IDs (columns) with the number of reads mapping to each contig in each sample. However, instead of the number of reads, we have the sequencing depth i.e. the number of bases divided by the contig length

This is calculated using MetaBAT2's `jgi_summarize_bam_contig_depths --outputDepth`. The values correspond to `(sum of exactly aligned bases) / ((contig length)-2*75)`. For example, for two reads aligned exactly with `10` and `9` bases on a 1000 bp long contig the depth is calculated by `(10+9)/(1000-2*75)` (1000bp length of contig minus 75bp from each end, which is excluded)


In [19]:
# Load the contig depths
contig_depths = decompress_gz(contig_depths_gz)
contig_depths_df = pd.read_csv(contig_depths, sep='\t')

# Preprocess the contig depths dataframe
contig_depths_df.index = contig_depths_df.contigName.str.split('_').str[1].astype(int)
contig_depths_df = contig_depths_df.drop(['contigName', 'contigLen'], axis=1)

# Drop all columns containing 'var' (i.e. the columns containing the variance)
contig_depths_df = contig_depths_df.drop([col for col in contig_depths_df.columns if 'var' in col], axis=1)

# Rename the columns e.g. from `SPAdes-group-HTP-SRR14530771.bam` to `SRR14530771`
contig_depths_df.columns = [col.split('-')[-1].split('.')[0] for col in contig_depths_df.columns]


In [20]:
# Merge the contig depths with the main dataframe
df = pd.merge(df, contig_depths_df, left_index=True, right_index=True, how='left')
df

,length,coverage,seq,bin_id,chromosome_score,plasmid_score,virus_score,classification,classification_method,"other_related_references(genome_id,species_name,radius,ANI,AF)",...,SRR14530884,SRR14530889,SRR14530886,SRR14530887,SRR14530882,SRR14530888,SRR14530881,SRR14530885,SRR14530880,SRR14530890
1,429168,132.214891,GACACCAGCAAGCGTCCGCCAAGCAAGCTTCTACGATCAAAAACTT...,002,0.9354,0.0472,0.0175,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",...,0.003212,0.005531,0.000909,0.038602,0.006647,0.254094,0.574263,0.006380,0.642186,0.671867
2,214794,155.876023,CATTGGGCCCGGGGGGGGGCCACGCCCCCCCCAACCAAAAAACCGA...,002,0.9492,0.0370,0.0138,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",...,0.010630,0.014661,0.003252,0.079698,0.020139,0.585271,1.626010,0.030781,2.068700,2.550200
3,204790,151.550375,GCCTGAACCTGAACATGCCGCTGTCCGATGAAATGAAGGCCTTGTA...,002,0.9312,0.0504,0.0184,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",...,0.005272,0.005331,0.001319,0.058682,0.007325,0.378401,0.998138,0.013629,1.166370,1.189980
4,185392,139.905194,CCGTCAAAGAGCATCTGGTTCAGGCTCAGCACGTAGCCGCTTCGGT...,002,0.9466,0.0389,0.0145,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",...,0.005738,0.004599,0.000308,0.054695,0.004615,0.297314,0.770735,0.009625,0.915712,0.930388
5,148022,144.176302,TGTCGCATGCGACAAGATCGGTACTCCCACAGGGGACTTTCACCCC...,002,0.9232,0.0566,0.0202,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,ANI,"GCF_009469615.1, s__Azonexus sp009469615, 95.0...",...,0.002001,0.009427,0.000541,0.038769,0.005531,0.373289,1.060360,0.011267,1.016190,1.282660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164662,78,2.000000,GGATCTGGTTTGTAATCTGGAATAGCATCGGGATCGCGCAGCCTTC...,tooshort,0.2063,0.1395,0.6542,NaN,NaN,NaN,...,0.000000,0.000000,2.615380,0.000000,0.000000,0.000000,0.000000,0.525641,0.000000,0.602564
164663,78,2.000000,TCTCTGGACAAAGTGTCCGCAAATGATTCAATCAGCATCCGCATTC...,tooshort,0.1568,0.1156,0.7276,NaN,NaN,NaN,...,0.000000,1.166670,0.000000,0.000000,0.000000,2.371790,0.000000,0.000000,0.000000,0.371795
164664,78,2.000000,CAATTAGTACTGGTTAGCTTAACGCATTACTGCGCTTCCACACCCA...,tooshort,0.4122,0.1507,0.4371,NaN,NaN,NaN,...,12.269200,5.089740,13.397400,14.961500,16.000000,3.358970,6.974360,4.794870,15.333300,7.166670
164665,77,327.824675,CGTAGTCTTACTCCAGCACCATGCCTAGAAAGAATCAACGCAAATC...,tooshort,0.1840,0.1365,0.6795,NaN,NaN,NaN,...,1.675320,3.376620,0.000000,4.064940,2.415580,0.532468,8.051950,0.000000,11.831200,8.389610
